## LAB 03

In [73]:
import numpy as np
import heapq

PUZZLE_DIM = 4

RANDOMIZE_STEPS = 100_000

class state:

    def __init__(self, t = None, p = None, parent = None, g = 0):
        if(t is None):
            self.tales = np.arange(0, PUZZLE_DIM**2)
            self.positions = np.arange(0, PUZZLE_DIM**2)
            self.h = 0
        else:
            self.tales = t
            self.positions = p
            self.update_h()

        self.g = g
        self.parent = parent

    def randomize(self):
        cur_state = self
        for i in range(RANDOMIZE_STEPS):
            ns = cur_state.getNeighbors()
            cur_state = ns[np.random.randint(0, len(ns))]

        self.tales = cur_state.tales
        self.positions = cur_state.positions
        self.h = cur_state.h

    def __hash__(self):
        return hash(tuple(self.tales))
    
    def __eq__(self, value):
        return np.array_equal(self.tales, value.tales)
    
    def __lt__(self, other):
        return (self.g + self.h) < (other.g + other.h)
    
    def genNeighborSwap(self, p1, p2):
        def swap(p1, p2, t, p):
            a = int(p[p1])
            b = int(p[p2])
            p[p1] = b
            p[p2] = a
            t[a] = p2
            t[b] = p1

        t = np.copy(self.tales)
        p = np.copy(self.positions)
        swap(p1, p2, t, p)
        return state(t, p, self, self.g + 1)
        
    def getNeighbors(self):
        neighbors = []
        blank_pos = self.tales[PUZZLE_DIM-1]

        #up
        if(blank_pos >= PUZZLE_DIM):
            p2 = blank_pos - PUZZLE_DIM
            neighbors += [self.genNeighborSwap(blank_pos, p2)]
            
        #down
        if(blank_pos < PUZZLE_DIM * (PUZZLE_DIM-1)):
            p2 = blank_pos + PUZZLE_DIM
            neighbors += [self.genNeighborSwap(blank_pos, p2)]
            
        #left
        if(blank_pos % PUZZLE_DIM != 0):
            p2 = blank_pos - 1
            neighbors += [self.genNeighborSwap(blank_pos, p2)]

        #right
        if(blank_pos % PUZZLE_DIM != PUZZLE_DIM -1):
            p2 = blank_pos + 1
            neighbors += [self.genNeighborSwap(blank_pos, p2)]

        return neighbors

    
    def isSolution(self):
        return self.h == 0
    
    
    def update_h(self):
        sum = 0
        for i in range(0, PUZZLE_DIM**2):
            p = self.tales[i]

            sum += abs(i%PUZZLE_DIM - p%PUZZLE_DIM) + abs(i // PUZZLE_DIM - p // PUZZLE_DIM)

        self.h = sum

In [74]:
def getPath(state):
    path = [state]

    while(state.parent is not None):
        state = state.parent
        path += [state]
    
    return path[::-1]

def a_star(initial_state):

    frontier_heap = []
    frontier_dict = dict()

    heapq.heappush(frontier_heap, initial_state)
    frontier_dict[initial_state] = initial_state
    seenStates = 1 

    while frontier_heap:
        current = heapq.heappop(frontier_heap)
        del frontier_dict[current]

        if current.isSolution():
            return (getPath(current), seenStates)
        
        neighbors = current.getNeighbors()
        seenStates += len(neighbors)
        for n in neighbors:
            old = frontier_dict.get(n, None)
            if old is None:
                heapq.heappush(frontier_heap, n)
                frontier_dict[n] = n
            elif old.g > n.g:
                old.parent = current
                old.g = n.g

initial = state()
initial.randomize()
path, totCost = a_star(initial)
print("path len: ", len(path), " n. of evaluated states: ", totCost)

path len:  55  n. of evaluated states:  10749935
